In [1]:
import os
import torch
import cv2
import torchvision.transforms as transforms
from torch.utils.data import Dataset , DataLoader, random_split
from dotenv import load_dotenv
import numpy as np
import torch.nn as nn
import torch.optim as optim
from imgDataset import imgDataset , custom_collate
from EAST import EAST
from train import train_model

s shape:  torch.Size([1, 16, 180, 320])
f1 shape:  torch.Size([1, 64, 90, 160])
f2 shape:  torch.Size([1, 128, 45, 80])
f3 shape:  torch.Size([1, 256, 22, 40])
f4 shape:  torch.Size([1, 512, 11, 20])
h1 shape unpooled:  torch.Size([1, 512, 22, 40])
f3 shape:  torch.Size([1, 256, 22, 40])
torch.Size([1, 768, 22, 40])
h2 :  torch.Size([1, 128, 22, 40])
h2 shape unpooled:  torch.Size([1, 128, 45, 80])
f2 shape:  torch.Size([1, 128, 45, 80])
torch.Size([1, 256, 45, 80])
h3 shape unpooled:  torch.Size([1, 32, 90, 160])
f1 shape:  torch.Size([1, 64, 90, 160])
score shape:  torch.Size([1, 1, 360, 640])
tensor([[[[0.4578, 0.4583, 0.4588,  ..., 0.4618, 0.4619, 0.4620],
          [0.4575, 0.4580, 0.4585,  ..., 0.4620, 0.4621, 0.4622],
          [0.4572, 0.4577, 0.4582,  ..., 0.4622, 0.4623, 0.4624],
          ...,
          [0.4596, 0.4601, 0.4607,  ..., 0.4632, 0.4630, 0.4628],
          [0.4603, 0.4606, 0.4610,  ..., 0.4627, 0.4627, 0.4627],
          [0.4609, 0.4611, 0.4613,  ..., 0.4623, 0.4

In [2]:
load_dotenv()
directory_train = os.getenv('Directory_train')
directory_train_textAndCoords = os.getenv('directory_train_textAndCoords')
directory_test = os.getenv('Directory_test')
directory_test_textAndCoords = os.getenv('directory_test_textAndCoords')


In [3]:
# Define path_imgs and path_labels
path_imgs = os.listdir(directory_train)
path_labels = os.listdir(directory_train_textAndCoords)

# Generate paths for training images and labels
train_img_paths = [os.path.join(directory_train, f) for f in path_imgs]
train_label_paths = [os.path.join(directory_train_textAndCoords, f) for f in path_labels]

#print(train_img_paths)
#print(train_label_paths)

In [4]:
train_dataset = [train_img_paths, train_label_paths]
train_dataset = imgDataset(train_img_paths, train_label_paths)
img, labels, coords = train_dataset[3]

print(f"Image shape: {img.shape}") # Should be [1, 360, 360] since it's grayscale
print(f"First Coords: {coords}")
print(f"First Coords:{len(coords)}")

'''max_len = max(len(train_dataset[j][2]) for j in range(len(train_dataset)))
pad_len = max_len - len(train_dataset[3][2])
print(pad_len)'''

batch = custom_collate(train_dataset)






Image shape: torch.Size([1, 360, 640])
First Coords: [[243, 108, 277, 105, 279, 121, 244, 124], [0, 180, 43, 177, 45, 192, 0, 200], [24, 197, 31, 196, 32, 203, 25, 205], [132, 117, 143, 113, 147, 121, 133, 127]]
First Coords:4


TypeError: expected Tensor as element 0 in argument 0, but got list

In [ ]:
print(batch[2][0])

In [ ]:
#spltining the dataset into training and validation sets
val_split = 0.2
training_size = int((1 - val_split) * len(train_dataset))
val_size = len(train_dataset) - training_size
training_dataset, val_dataset = random_split(train_dataset, [training_size, val_size])
loader_train = DataLoader(training_dataset, batch_size=32, shuffle=True, collate_fn = custom_collate)
loader_val = DataLoader(val_dataset, batch_size=32, shuffle=True, collate_fn = custom_collate)

#print(f"loader_train: {loader_train}")
#print(f"loader_val: {loader_val}")

In [ ]:
model = EAST(color_channel=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
#loasding model if it exists
if os.getenv('Load_model') == 'True': 
    model.load_state_dict(torch.load(os.getenv('model_path')))
    print("Model loaded successfully")
else:
    print("Model not load, starting from scratch")

In [ ]:
#defualt model otipions
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
cycles = 12
train_model(model = model, loader_train = loader_train, loader_val = loader_val, criterion = criterion, optimizer = optimizer, cycles = cycles)

In [ ]:
model_save_path = os.getenv('Model_save_path', 'east_model.pth')
torch.save(model.state_dict(), model_save_path)